In [1]:
import os
import numpy as np
from utils import *
import pandas as pd

In [2]:
# from batch ID to the corresponding sample IDs
skip_samples = {
    5: range(40, 50), 
    10: range(10,20),
    11: range(10),
    12: range(20, 30), 
    15: range(40, 50)
}

annotator_list = {
    7: ['yujia', 'manveer'],
    8: ['miaoran', 'chenyu'],
    10: ['erana', 'vivek', 'manveer'],
    11: ['rogger', 'matt'], #,'matt', , 'new', 'yujia'
    13: ['erana', 'miaoran'], #['erana', 'weisi', 'miaoran']
    16: ['miaoran', 'matt'] #['miaoran', 'yujia', 'matt', 'weisi', 'new']
}
# batch 16
# ['yujia', 'matt']: 0.299	0.267	0.209	0.294	0.244	0.340	0.330
# ['yujia', 'weisi']: -0.417	-0.376	-0.333	-0.384	-0.320	-0.391	-0.387
# ['matt', 'weisi']: -0.138	-0.117	-0.074	-0.104	-0.065	-0.147	-0.092
exclude_batch = []
# exclude_batch = [11,13,16]

In [3]:
agreement_results = {}
span_agreement_results = {}
result_path = 'batch_5_src_no_sports/results'
label_maps = {
    'c-q-ub':{'consistent':2, 'questionable':1, 'unwanted':0, 'benign': 0},
    'c-q+b-u': {'consistent':2, 'questionable':1, 'benign': 1, 'unwanted':0},
    'c+b-q-u':{'consistent':2, 'benign': 2, 'questionable':1, 'unwanted':0},
    'c-b-u':{'consistent':2, 'benign': 1, 'questionable':np.nan, 'unwanted':0},
    'c+b-q+u':{'consistent':1, 'benign': 1, 'questionable':0, 'unwanted':0},
    'c-b+q+u':{'consistent':1, 'benign': 0, 'questionable':0, 'unwanted':0},
    'c-u':{'consistent':1, 'benign': np.nan, 'questionable':np.nan, 'unwanted':0},
}
for batch_id in range(1,16+1):
    if batch_id in exclude_batch:
        continue
    file_path = os.path.join(result_path, f"batch_{batch_id}_annotation.json")
    skip_sample_ids = []
    if batch_id in skip_samples:
        skip_sample_ids = [str(s_id) for s_id in skip_samples[batch_id]]
        print (f"Skipping samples {skip_sample_ids}")
    selected_annotators = None
    # there is an unexpected "new" annotator in batch 7
    if batch_id in annotator_list:
        selected_annotators = annotator_list[batch_id]
        
    agreement_local_batch = {}
    span_agreement_local_batch = {}
    for short_name, label_map in label_maps.items():
        agreement_num = compute_interannotator_agreement(file_path=file_path, summary_sent_file='summary_sent_list.jsonl', label_map=label_map, selected_annotators=selected_annotators, skip_sample_ids=skip_sample_ids)
        agreement_local_batch[short_name] = agreement_num
        span_level_agreement_num = compute_span_level_interannotator_agreement(file_path=file_path, summary_sent_file='summary_sent_list.jsonl', label_map=label_map, selected_annotators=selected_annotators, skip_sample_ids=skip_sample_ids)
        span_agreement_local_batch[short_name] = span_level_agreement_num
        
    agreement_results[batch_id] = agreement_local_batch
    span_agreement_results[batch_id] = span_agreement_local_batch

annotator for agreement computation: ['forrest', 'renyi']
disagreed sample ids in batch: [14, 15, 16, 17, 19, 23]
forrest [0, 0, 0, 0, 0, 0]
renyi [2, 2, 2, 2, 2, 2]
interval Krippendorff's alpha for label map
{'consistent': 2, 'questionable': 1, 'unwanted': 0, 'benign': 0}
0.759
compute span level interannotator agreement
batch_5_src_no_sports/results/batch_1_annotation.json
annotator for span level agreement computation: ['forrest', 'renyi']
84
84
{'forrest': [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1], 'renyi': [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]}
disagreed sent id

In [4]:
# pd display only 3 digits
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame.from_dict(agreement_results, orient='index')
df

,c-q-ub,c-q+b-u,c+b-q-u,c-b-u,c+b-q+u,c-b+q+u,c-u
1,0.759,0.858,0.870,0.858,0.841,0.759,0.908
2,0.610,0.665,0.593,0.724,0.600,0.639,0.824
3,0.571,0.596,0.474,0.622,0.417,0.569,0.666
4,0.660,0.616,0.524,0.643,0.520,0.656,0.671
5,0.493,0.499,0.429,0.613,0.308,0.396,0.686
6,0.645,0.707,0.663,0.732,0.603,0.625,0.790
7,0.878,0.653,0.309,0.747,0.320,0.891,1.000
8,0.761,0.778,0.778,0.844,0.824,0.861,0.844
9,0.441,0.563,0.564,0.598,0.522,0.382,0.683
10,0.409,0.611,0.636,0.672,0.434,0.338,0.762


In [5]:
df.mean()

c-q-ub    0.577
c-q+b-u   0.624
c+b-q-u   0.591
c-b-u     0.683
c+b-q+u   0.561
c-b+q+u   0.591
c-u       0.756
dtype: float64

In [6]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame.from_dict(span_agreement_results, orient='index')
df

,c-q-ub,c-q+b-u,c+b-q-u,c-b-u,c+b-q+u,c-b+q+u,c-u
1,0.621,0.753,0.739,0.751,0.704,1.000,1.000
2,0.525,0.524,0.564,0.592,0.531,1.000,1.000
3,0.528,0.473,0.473,0.471,0.448,1.000,1.000
4,0.514,0.523,0.525,0.571,0.417,1.000,1.000
5,0.474,0.410,0.398,0.448,0.319,1.000,1.000
6,0.634,0.656,0.670,0.683,0.654,1.000,1.000
7,0.788,0.471,0.459,0.495,0.387,1.000,1.000
8,0.629,0.689,0.714,0.756,0.673,1.000,1.000
9,0.714,0.572,0.569,0.571,0.569,1.000,1.000
10,0.424,0.536,0.588,0.629,0.519,1.000,1.000
